<a href="https://colab.research.google.com/github/okj3031402/LLM_study/blob/main/building_a_llm_math_chain_with_LCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 수학체인

사용자 질문에 대해 수학 함수를 호출할 수 있도록 인자를 만들어주는 체인

In [ ]:
!pip install langchain
!pip install openai
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import openai

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## 수학 공식 푸는 패키지 : numexpr

In [ ]:
import numexpr

In [ ]:
numexpr.evaluate('15**0.3432')

array(2.53299609)

## 자연어로 질문하면 알아서 수학공식을 만들어서, numexpr로 넘겨주면 어떨까?

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# numexpr.evaluate("수학 공식") 수식을 실행하는 파이썬 패키지
# 자연어 > llm > 수학 공식 > 파싱 > numexpr.evaluate
# 출력형식
'''
```text
13**0.3432
```
'''

system_content_template = """You are a helpful assistant that make an only ARGUMENT for numexpr.evaluate according to user query such as
```text
13**0.3432
```
when user query is 'What is 13 raised to the .3432 power?'

Please response as code block using text back quote like
```text
13**0.3432
```
without other description."""

human_content_template = "{user_input}"

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_content_template),
    ("human", human_content_template),
])

chat_model = ChatOpenAI(temperature=0)

chain = chat_prompt_template | chat_model

chain.invoke({"user_input" : "What is 15 raised to the .3432 power?"})

AIMessage(content='```text\n15**0.3432\n```', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 101, 'total_tokens': 112}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-de6a1497-5d06-4902-8ef0-cead8b75edac-0')

In [ ]:
from langchain.schema import BaseOutputParser

# ```text\n15**0.3432\n``` > "15**0.3432"

class CodeOutputParser(BaseOutputParser):
    def parse(self, text: str):
        value = text.strip().split("```")
        print(value)
        return value[1].split("\n")[1]

chain = chat_prompt_template | chat_model | CodeOutputParser()

output = chain.invoke({"user_input" : "What is 15 raised to the .3432 power?"})

print(output)

['', 'text\n15**0.3432\n', '']
15**0.3432


In [ ]:
chain = chat_prompt_template | chat_model | CodeOutputParser() | numexpr.evaluate

output = chain.invoke({"user_input" : "What is 15 raised to the .3432 power?"})

print(output)

['', 'text\n15**0.3432\n', '']
2.5329960941138205


In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

chain = ({"user_input" : RunnablePassthrough() }
        | chat_prompt_template
        | chat_model
        | CodeOutputParser()
        | numexpr.evaluate)

output = chain.invoke("What is 15 raised to the .3432 power?")

print(output)

['', 'text\n15**0.3432\n', '']
2.5329960941138205
